Copyright 2022 Masamichi J. Miyama

In [11]:
import numpy as np
import optuna
from tqdm import tqdm

最もシンプルな使い方

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/001_first.html

In [12]:
# 目的関数の定義
def objective(trial):
    # -10から10までの値を取るfloat変数の定義（最適化対象のパラメタ）
    # hyperoptと異なるのはオラクルたる目的関数側で変数定義をやる点？
    x = trial.suggest_float("x", -10, 10)

    return (x - 2) ** 2

上の例では`float`型のパラメタを扱ったがそれ以外にも整数値、カテゴリ変数など色々使える。探索空間の指定の仕方等、詳しくは下記URLを参照

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/002_configurations.html

In [13]:
# 最適化プロジェクトの定義
study = optuna.create_study()
# 最適化実行
study.optimize(objective, n_trials=100)

[I 2022-06-19 00:08:05,615] A new study created in memory with name: no-name-970d9cf6-6e89-4e6c-8fe6-7ebe0648ae4b
[I 2022-06-19 00:08:05,617] Trial 0 finished with value: 123.70723061162384 and parameters: {'x': -9.122375223468405}. Best is trial 0 with value: 123.70723061162384.
[I 2022-06-19 00:08:05,618] Trial 1 finished with value: 20.023617635649824 and parameters: {'x': -2.4747757078595374}. Best is trial 1 with value: 20.023617635649824.
[I 2022-06-19 00:08:05,618] Trial 2 finished with value: 54.579494590416715 and parameters: {'x': -5.387793621265873}. Best is trial 1 with value: 20.023617635649824.
[I 2022-06-19 00:08:05,619] Trial 3 finished with value: 68.69620909171522 and parameters: {'x': -6.288317627342428}. Best is trial 1 with value: 20.023617635649824.
[I 2022-06-19 00:08:05,620] Trial 4 finished with value: 44.27785904125987 and parameters: {'x': -4.65416103211065}. Best is trial 1 with value: 20.023617635649824.
[I 2022-06-19 00:08:05,621] Trial 5 finished with val

In [22]:
# 最小値の表示
best_params = study.best_params
found_x = best_params["x"]
print(f"Found x: {found_x}, (x - 2)^2: {(found_x - 2) ** 2}")

Found x: 0.004240863732467695, (x - 2)^2: 3.9830545299953264


探索の経過の可視化 (**Plotlyパッケージをインストール必要**)

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html

In [15]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

バッチ最適化

基本的にはAsk-and-Tellインターフェースという対話的な利用の一ユースケースとしてバッチ最適化がある。たとえば、観山が困っていたパラメタ側に関する制約を守らせるためのサンプリングを事前にして、満たす入力値とこれに対する評価値のみをOptunaに伝えたいときもおそらくこの対話インターフェースを使えばよさそう。

https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/009_ask_and_tell.html

In [16]:
# バッチでパラメタを受け取って値を返す目的関数
# バッチ最適化というか対話的にOptunaを使う場合には目的関数内でパラメタを定義しないみたい
def batched_objective(xs: np.ndarray, ys: np.ndarray):
    return xs ** 2 + ys

In [23]:
# バッチサイズ
batch_size = 10
# 問い合わせ回数
num_query = 10
study = optuna.create_study()

for _ in tqdm(range(num_query)):
    # create batch
    trial_ids = []
    x_batch = []
    y_batch = []

    # バッチサイズ分だけデータを作る
    for _ in range(batch_size):
        # Optunaにぶち込むためのデータ形式を作る
        trial = study.ask()
        trial_ids.append(trial.number)
        # ここでパラメタの定義と探索空間を指定
        x_batch.append(trial.suggest_float("x", -10, 10))
        y_batch.append(trial.suggest_float("y", -10, 10))

    # 現在のバッチに対する評価値をオラクルに問い合わせる
    x_batch = np.array(x_batch)
    y_batch = np.array(y_batch)
    objectives = batched_objective(x_batch, y_batch)

    # 問い合わせ結果をOptunaに伝える
    for trial_id, objective in zip(trial_ids, objectives):
        study.tell(trial_id, objective)

[I 2022-06-19 00:14:53,273] A new study created in memory with name: no-name-323a7932-a2bd-48ac-b63e-19707dd2484f
100%|██████████| 10/10 [00:00<00:00, 20.34it/s]


In [24]:
# 最小値の表示
best_params = study.best_params
found_x = best_params["x"]
found_y = best_params["y"]
print(f"Found x: {found_x}, y: {found_y}, objective: {batched_objective(found_x, found_y)}")

Found x: 0.22177243177503503, y: -9.908083700605765, objective: -9.858900689110353


In [25]:
# 10個ずつ10回聞いたのでトライアル合計数としては100個分となる
# 割と1回の問い合わせバッチの中身が偏っている印象を受けるがアルゴリズムの選択を変えると挙動が変わるのかもしれない
plot_optimization_history(study)